<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/getScreenerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import openpyxl

from google.colab import drive
drive.mount('/content/drive')

from google.colab import data_table

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Try-Hyperlink.xlsx')


def _get_link_if_exists(cell) -> str | None:
    try:
        return cell.hyperlink.target
    except AttributeError:
        return None


def extract_hyperlinks_from_xlsx(
    file_name: str, sheet_name: str, columns_to_parse: list[str], row_header: int = 1
) -> pd.DataFrame:
    df = pd.read_excel(file_name, sheet_name)
    ws = openpyxl.load_workbook(file_name)[sheet_name]
    for column in columns_to_parse:
        row_offset = row_header + 1
        column_index = list(df.columns).index(column) + 1
        df[column] = [
            _get_link_if_exists(ws.cell(row=row_offset + i, column=column_index))
            for i in range(len(df[column]))
        ]
    return df

df_data = extract_hyperlinks_from_xlsx('/content/drive/My Drive/data/stocks/myScreenerDB.xlsx','data', ['Name'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_data['Symbol'] = df_data['Name'].str.split('company/').str[1].str.split('/').str[0]
df_data.columns = ['Rank', 'Link', 'Close','PE','MCap','ROCE%','ROE%','MedPE','5YrROCE%','5YsROE%','Debt2EqR','5YrSalesG%','5YrProfitG%','OPM%','QtrPAT','PB','5YrMCap','CapType', 'Symbol']

In [3]:
df_data.to_csv('/content/drive/My Drive/data/stocks/myScreenerDB.csv', index=False)

In [4]:
cols = ['Symbol','PE','MCap','ROCE%','ROE%','MedPE','5YrROCE%','5YsROE%','Debt2EqR','5YrSalesG%','5YrProfitG%','OPM%','QtrPAT','PB','5YrMCap','CapType']
df = df_data[cols]

In [5]:
df_tmp = df[(df['PE'] < df['MedPE']) & (df['Debt2EqR'] < .25) & (df['5YrSalesG%'] > 15) & (df['5YrProfitG%'] > 25)]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,PE,MCap,ROCE%,ROE%,MedPE,5YrROCE%,5YsROE%,Debt2EqR,5YrSalesG%,5YrProfitG%,OPM%,QtrPAT,PB,5YrMCap,CapType
106,IRCTC,66.22,75252.00,59.19,45.36,69.62,46.46,35.11,0.02,19.29,34.88,35.00,310.47,26.44,NaN,MC
141,PIIND,34.73,55305.81,21.69,18.46,47.18,21.73,17.54,0.02,23.31,27.72,25.57,448.60,6.93,12207.55,MC
199,SONACOMS,72.36,36001.66,22.05,18.29,80.29,27.04,21.99,0.16,34.33,38.32,28.10,132.71,14.80,NaN,MC
331,INDIAMART,49.84,16992.01,17.47,13.93,56.24,41.36,NaN,0.03,18.40,42.13,28.32,91.60,10.78,NaN,SC
336,AFFLE,59.84,16286.48,20.20,18.51,66.79,47.65,27.36,0.14,53.69,54.45,19.56,76.82,10.22,NaN,SC
362,CLEAN,58.18,14793.63,44.50,33.20,67.30,53.98,38.07,0.00,31.16,43.28,43.88,62.63,13.57,NaN,SC
366,BASF,30.82,14575.96,20.12,15.19,34.35,16.69,14.38,0.04,19.57,25.93,5.68,140.06,4.98,8325.34,SC
392,TANLA,24.85,13376.14,37.91,31.18,29.83,25.51,25.17,0.05,33.48,92.63,19.65,140.13,7.65,343.45,SC
399,FINEORG,29.56,13206.78,65.34,49.43,41.73,40.32,34.37,0.01,28.71,45.35,27.14,94.21,7.71,NaN,SC
435,AETHER,88.62,11466.92,18.44,15.91,90.80,28.27,28.50,0.01,43.05,75.61,27.34,23.83,5.60,NaN,SC
